## Transactions - Homework
#### 03.6 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-16

* Homework 02 of the WSC class

## Setup
Starting with this chapter 3.4, the lines below will always automatically load ...
* The accounts MyAlgo, Alice, Bob, Charlie, Dina
* The Purestake credentials
* The functions in `algo_util.py`

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn, MultisigTransaction
import algosdk.error
import json

In [ ]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

### Connecting to the Algorand Mainnet
Changed `algod_address=...` to `cred['algod_main']`

In [ ]:
# Initialize the algod client (Testnet or Mainnet) and check
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_main'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

### Exercise 3.1

In [ ]:
# Get holdings of testnet Algos
address = "WSC234WDSXHLWICSZ5DVJNRZLJHHK23D3ZSP6MS2CJNII4ZDQTC367VUN4"
course_money = algod_client.account_info(address)["amount"]/1e6
print(course_money)

## Exercise 3.2.a

In [ ]:
# Step 1: prepare and create unsigned transaction
sp       = algod_client.suggested_params()       # suggested params
amount   = 0.5
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

txn = PaymentTxn(sender = MyAlgo['public'],     # <--- From
                 sp = sp, 
                 receiver = Bob['public'],      # <---- To
                 amt = amt_microalgo            # <---- Amount in Micro-ALGOs
                )

# Step 2: sign
stxn = txn.sign(MyAlgo['private']) 

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

### Exercise 3.2.b

In [ ]:
# Step 1: Prepare
sp       = algod_client.suggested_params()       # suggested params
amount    = 0.5
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)
# subtract fee and min holding requirement for Bob
amt_microalgo = amt_microalgo - sp.min_fee - int(0.1*algo_precision)

# Step 1b: The note
note_txt  = "Course account: "+ str(course_money)
note_byte = note_txt.encode()

txn = PaymentTxn(sender=Bob['public'],
                 sp=sp, 
                 receiver = MyAlgo['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )

# Step 2: sign TX
stxn = txn.sign(Bob['private'])

# Step 3: send TX
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)